In [80]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import os,json,base64,io
import pydicom
import pandas as pd
from io import BytesIO,StringIO
import base64
from PIL import Image
import numpy as np
from ipydatagrid import DataGrid


from IPython.display import display
from ipywidgets import widgets, Output

classification_dict={}

# Function to handle the upload of the JSON file
def upload_json(change):
    global ontology_types

    try:        
        # Check if there is an uploaded file
        if upload_button.value:
            # Get the file info for the first uploaded file
            file_info = upload_button.value[-1]
            file_content = file_info['content']
    
            # Convert memoryview to bytes, then decode to string
            json_str = bytes(file_content).decode('utf-8')
    
            # Parse JSON string manually
            parsed_json = json.loads(json_str)
    
            # Assuming the JSON is a list of types
            ontology_types = parsed_json['scan_types'] if isinstance(parsed_json['scan_types'], list) else []
    
            ontology_dropdown.options = ontology_types
    except Exception as e:
        display(f"Error: {e}")

# Function to handle the upload of the JSON file
def upload_file_classes(change):
    global ontology_types
    try:
        
        # Check if there is an uploaded file
        if upload_button.value:
            # Get the file info for the first uploaded file
            file_info = upload_button.value[-1]
            file_content = file_info['content']
    
            # Convert memoryview to bytes, then decode to string
            json_str = bytes(file_content).decode('utf-8')
    
            # Parse JSON string manually
            parsed_json = json.loads(json_str)
    
            # Assuming the JSON is a list of types
            ontology_types = parsed_json['scan_types'] if isinstance(parsed_json['scan_types'], list) else []
    
            ontology_dropdown.options = ontology_types
    except Exception as e:
        display(f"Error: {e}")



# Function to handle the collection of DICOM files
def collect_files(b):    
    #print('collecting files')
    global dicom_files, current_index
    root_dir = text_input.value

    for root, dirs, files in os.walk(root_dir):
        # Filter and sort DICOM files in each directory
        dicom_files_in_dir = [f for f in files if is_dicom_file(os.path.join(root, f))]
        if dicom_files_in_dir:
            dicom_files_in_dir.sort()
            middle_file = dicom_files_in_dir[len(dicom_files_in_dir) // 2]  # Pick the middle file
            yield os.path.join(root, middle_file)


def is_dicom_file(file_path):
    try:
        # Attempt to read only the file's metadata
        pydicom.dcmread(file_path, stop_before_pixels=True)
        return True
    except:
        return False


def get_file_classification_data(file_path):
    ind=classification_dict['files'].index(file_path)
    out_dict={}
    if ind<0: return out_dict
        
    for key in classification_dict.keys():
        out_dict[key]=classification_dict[key][ind]

    #set the dropdown value.
    if 'labels1' in out_dict:
        label1=classification_dict['labels1'][ind]
        if label1 in ontology_dropdown.options:
            ontology_dropdown.value=label1
    
    return out_dict

def format_file_classification_string(d):
    str=''
    if not d: return str;
    str+="L1: {}, p={:0.2f}\n".format(d['labels1'],d['probs1'])
    str+="L2: {}, p={:0.2f}\n".format(d['labels2'],d['probs2'])
    str+="GI: {:0.2f}, MC: {:0.2f}".format(d['pred_gini_impurity'],d['pred_margin_confidence'])
    return str
    
# Function to display the DICOM file
def display_file(file_path):    
    global file_prefix
    try:        
        image_panel.clear_output(wait=False)
        metadata_text1.value=""
        metadata_text2.value=""
        
        with open(file_path, 'rb') as f:
            ds = pydicom.dcmread(f)
            prioritized_tags = ['SeriesDescription', 'ImageType', 'PixelSpacing','SliceThickness']
            metadata_str = ''
            for tag in prioritized_tags:
                if tag in ds:
                    metadata_str += f"{tag}: {ds[tag].value}\n"
    
            # Add the rest of the tags, excluding PixelData
            metadata_str += "\n"+"\n".join([f"{tag}: {ds[tag].value}" 
                                       for tag in ds.dir() 
                                       if hasattr(ds[tag], 'value') and tag not in ['PixelData'] + prioritized_tags])
    
            metadata_text1.value = metadata_str

            #now populate classification panel, if classification is provided.                      
            truncated_path=file_path[file_path.startswith(file_prefix) and len(file_prefix):]
            metadata_text2.value = format_file_classification_string(get_file_classification_data(truncated_path))

            ds.decompress()
            # Process the DICOM image
            image = ds.pixel_array
    
            # Handle multi-dimensional images (e.g., 3D or color images)
            sh=image.shape
            if image.ndim == 4:
                # Taking the first slice for simplicity
                image=image[sh[2]//2][sh[3]//2]
    
            elif image.ndim==3:
                image=image[sh[2]//2]
    
            # Normalize and convert to uint8
            image = image - np.min(image)
            if np.max(image) != 0:
                image = image / np.max(image) * 255
            image = image.astype(np.uint8)
    
            # Convert to PIL image
            pil_img = Image.fromarray(image)
    
            # Convert the PIL image to a byte buffer
            buffer = io.BytesIO()
            pil_img.save(buffer, format="JPEG")
            buffer.seek(0)
    
            # Display the image in the output widget
            with image_panel:
                #clear_output(wait=True)
                display(pil_img)
                #
                #image_widget = widgets.Image(value=buffer.read(), format='jpeg')
                #image_widget.layout.max_width = '400px'  # Set the maximum width
                #image_widget.layout.max_height = '400px'  # Set the maximum height
                #display(image_widget)
    except Exception as e:
        display(f"Image display error: {e}")


# Widgets

upload_button = widgets.FileUpload(description='Upload JSON', accept='.json', multiple=False)
upload_button.observe(upload_json, names='data')

upload_classification_button=widgets.FileUpload(description='Upload label', accept='.csv', multiple=False)
upload_classification_button_output=Output()

def upload_classification_file(change):
    global classification_dict
    global upload_classification_button
    global classification_dataframe
    
    classification_dict.clear()
    uc=upload_classification_button
    
    try:
        if uc.value:            
            file_content=uc.value[0]['content']
            object=BytesIO(file_content)
            df=pd.read_csv(object)

            # Loop through each column in the DataFrame and add it to the dictionary
            for column in df.columns:
                # Check the data type of the column and convert to list
                if df[column].dtype == 'object':  # For strings
                    classification_dict[column] = df[column].astype(str).tolist()
                else:  # For real numbers
                    classification_dict[column] = df[column].astype(float).tolist() 
            classification_dataframe=df
            #display_grid_interface(df)
            
    except Exception as e: 
        print(e)

upload_classification_button.observe(upload_classification_file, names='value')


text_input = widgets.Text(description='DICOM Dir:',value='/data/MIRRIR/archive/M19004_AS')
text_input.value='/data/MIRRIR/M19021_glioma2'
file_prefix=text_input.value+'/'

collect_button = widgets.Button(description='Collect Files')

collect_button_output=Output()
#@collect_button_output.capture(clear_output=True)
file_generator=collect_files(0)

@collect_button_output.capture(clear_output=True)
def collect_on_click(b):
    global classification_dataframe
    collect_button_output.clear_output()    
    #print("collect_on_click")
    upload_json(0)
    next_file(b)
    display_grid_interface(classification_dataframe)
    
collect_button.on_click(collect_on_click)

metadata_text1 = widgets.Textarea(description='DICOM:', layout=widgets.Layout(width='100%', height='300px'))
metadata_text2 = widgets.Textarea(description='Classification:', layout=widgets.Layout(width='100%', height='100px'))

image_panel = widgets.Output(layout=widgets.Layout(width='50%', height='400px'))
panel0=widgets.VBox([metadata_text1, metadata_text2],layout=widgets.Layout(width='50%', height='450px'))
panel = widgets.HBox([panel0, image_panel])

next_button = widgets.Button(description='Next')
next_button_output=Output()

# Function to handle "Next" button click
num_clicked=0

@next_button_output.capture(clear_output=True)
def next_file(b):
    global num_clicked    
    global dicom_files
    
    next_button_output.clear_output()   
    num_clicked+=1

    global file_generator,file_types
    try:
        file_path=next(file_generator)
        #print(file_path)
        display_file(file_path)
        dicom_files.append(file_path)
        file_types.append(ontology_dropdown.value)
        print("{}. {}".format(num_clicked,file_path))
    except StopIteration:
        display("No more files.")
    except Exception as e:
        print(f"Error: {e}".format(e))

next_button.on_click(next_file)
ontology_dropdown = widgets.Dropdown(description='Type:', options=[])
finish_button = widgets.Button(description='Finish')
finish_button_output = widgets.Output()

# Function to handle "Finish" button click
@finish_button_output.capture(clear_output=True)
def finish(b): 
    try:
        print('creating data frame')
        #padded_file_types = file_types + [None] * (len(dicom_files) - len(file_types))
    
        df = pd.DataFrame({'files': dicom_files,
                           'user_label': file_types})
        
        output_file = StringIO()
        df.to_csv(output_file, index=False)
        output_file.seek(0)
        print ('creating link')
        b64 = base64.b64encode(output_file.getvalue().encode()).decode()
        href = f'<a href="data:application/octet-stream;base64,{b64}" download="output.csv">Download CSV</a>'
        download_link.value = href
        print('created link')
        
    except Exception as e: 
        print("Error: ", e)

def display_grid_interface(df):
    #print("aaa")
    global grid, grid_output
    #global panel, next_button, ontology_dropdown, finish_button, download_link
    grid=DataGrid(df,editable=False)
    grid.layout.height="300px"
    grid.auto_fit_columns=True
    grid.on_cell_click(on_grid_cell_click)
    display(grid,grid_output)
#    display(panel)
#    display(next_button)
#    display(widgets.HBox([ontology_dropdown, finish_button]),finish_button_output)
#    display(download_link)

grid_output=widgets.Output()
@grid_output.capture(clear_output=True)
def on_grid_cell_click(event):
    #print("on_grid_cell_click")
    global classification_dataframe, text_input
    #metadata_text1.value=str(event)
    
    row=event['row']
    column = classification_dataframe.columns.get_loc('files')
    file=classification_dataframe.iloc[row,column]
    display_file(text_input.value+'/'+file)




finish_button.on_click(finish)
download_link = widgets.HTML()

# Display widgets
display(widgets.HBox([upload_button,upload_classification_button]))
#display(upload_classification_button,upload_classification_button_output)
display(widgets.HBox([text_input,collect_button]))
display(collect_button_output)

display(panel)
#display(next_button,next_button_output)
display(widgets.HBox([next_button,ontology_dropdown, finish_button]),finish_button_output)
display(download_link)

# Global variables
ontology_types = []
dicom_files = []
current_index = 0
file_types = []


Output()

Output()

HTML(value='')

In [74]:
data = {"Column 1": [1, 2, 3, 4,5], "Column 2": ["A", "B", "C", "D","E"]}
df=pd.DataFrame(data)
grid=DataGrid(df)
#b=widgets.Button(description='test')
#b.layout.width='100%'
#grid_box=widgets.Box([b])
#grid_box.layout.width='50%'
grid.layout.width='100%'
grid

DataGrid(auto_fit_params={'area': 'all', 'padding': 30, 'numCols': None}, corner_renderer=None, default_render…

In [100]:
import re

def parse_path(path, path_type):
    experiment_id = 'NA'
    scan_id = 'NA'
    
    if path_type == 'project':
        match = re.match(r'.*/([^/]+)/([^/]+)/SCANS/([^/]+)/DICOM/([^/]+)', path)
        if match:
            experiment_id = match.group(2)
            scan_id = match.group(3)
    elif path_type == 'experiment':
        match = re.match(r'.*/SCANS/([^/]+)/DICOM/([^/]+)', path)
        if match:
            scan_id = match.group(1)
    return experiment_id, scan_id

# Example usage:
path = '/SCANS/scan456/DICOM/iaad.mage.dcmdd'
path_type = 'scan'
experiment_id, scan_id = parse_path(path, path_type)
print("Experiment ID:", experiment_id)
print("Scan ID:", scan_id)


Experiment ID: NA
Scan ID: NA
